In [19]:
"""
Step 1 in HailPixel processing - subset orthomoasic
This notebook breaks a large png orthomosaic into smaller subsets with an overlap.
Also generates an index file to determine image coordinates of each subset.
"""

'\nThis notebook breaks a large png orthomosaic into smaller subsets with an overlap.\nAlso generates an index file to determine image coordinates of each subset\n'

In [20]:
import math
import tifffile
import numpy as np
from PIL import Image
import pandas as pd

In [27]:
#config
pass_str = 'pass5'

source_img_ffn = '/home/meso/Desktop/20200119 hail case/aerial_survey/{pass_str}/{pass_str}.tif'.format(pass_str=pass_str)
output_img_path = '/home/meso/Desktop/20200119 hail case/aerial_survey/{pass_str}/subsets'.format(pass_str=pass_str)
output_idx_ffn = '/home/meso/Desktop/20200119 hail case/aerial_survey/{pass_str}/index.csv'.format(pass_str=pass_str)

# subset size
subset_sz      = 500 #pixels
# edge overlap (total subset suze = subset_sz+(2*subset_overlap) x subset_sz+(2*subset_overlap))
subset_overlap = 50 #pixels

In [28]:
#read image
im    = tifffile.imread(source_img_ffn)
#extract size
im_sz = np.shape(im)
print('original image size', im_sz)
im_y  = im_sz[0]
im_x  = im_sz[1]
im_z  = im_sz[2]
# set padded image size as amultiple of subset_sz
im_pad_y = math.ceil(im_y / subset_sz) * subset_sz
im_pad_x = math.ceil(im_x / subset_sz) * subset_sz
# pad image (include overlaps)
pad_y0 = subset_overlap
pad_x0 = subset_overlap
pad_yn = subset_overlap + (im_pad_y-im_y)
pad_xn = subset_overlap + (im_pad_x-im_x)
im_padded = np.pad(im, ((pad_y0, pad_yn), (pad_x0, pad_xn), (0, 0)), mode='constant')
print('padded image size', np.shape(im_padded))

original image size (7720, 9379, 3)
padded image size (8100, 9600, 3)


In [29]:
#for generate subset image and build index file
subset_idx = 0
dict_idx = {'idx':[], 'x0':[], 'y0':[]}
for y in np.arange(subset_overlap, im_pad_y, subset_sz):
    for x in np.arange(subset_overlap, im_pad_x, subset_sz):
        #extract subset
        subset_y0 = y - subset_overlap
        subset_x0 = x - subset_overlap
        subset_yn = y + subset_sz + subset_overlap
        subset_xn = x + subset_sz + subset_overlap
        subset_im = im_padded[subset_y0:subset_yn,subset_x0:subset_xn,0:3]
        #write to file
        out_ffn = '/'.join([output_img_path, str(subset_idx).zfill(4)]) + '.png'
        im = Image.fromarray(subset_im)
        im.save(out_ffn)
        #append to dict
        dict_idx['idx'].append(subset_idx)
        dict_idx['y0'].append(subset_y0)
        dict_idx['x0'].append(subset_x0)
        #print('generated output for',out_ffn)
        subset_idx += 1
print('finished')

#write index dictionary to file
df = pd.DataFrame.from_dict(dict_idx)
df.to_csv(output_idx_ffn)

finished
